In [1]:
from chirp.models import Chirp, Tag
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from users.models import Profile

In [2]:
from django.core.urlresolvers import reverse
from django.db.models import Avg, Count, F, Max, Min, Sum, Q, Prefetch
from django.core.cache import cache
from django.conf import settings
from django.db import transaction
from django.utils import timezone

In [3]:
Chirp.objects.all()

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

In [4]:
type(Chirp.objects.all())

django.db.models.query.QuerySet

In [5]:
Chirp.objects.filter(id=1)

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

In [6]:
Chirp.objects.get(pk=1)

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

In [9]:
Chirp.objects.filter(author__username='adam')

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

In [11]:
Chirp.objects.filter(message__icontains='First')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

# Querysets are unique

In [7]:
q1 = Chirp.objects.filter(author__username='jeff')
q2 = q1.exclude(title__icontains='First')

# Quersets are Lazy

In [12]:
q = Chirp.objects.filter(author__username='jeff')
q = q.exclude(title__icontains='First')

In [14]:
q

[<Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>]

# Get

In [15]:
Chirp.objects.get(pk=1)

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

In [16]:
Chirp.objects.filter(id=1)[0]

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

In [17]:
Chirp.objects.get(pk=1000)

DoesNotExist: Chirp matching query does not exist.

In [18]:
Chirp.objects.get(author__username='jeff')

MultipleObjectsReturned: get() returned more than one Chirp -- it returned 5!

# Limiting Results

In [20]:
Chirp.objects.all()[:2]

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>]

In [22]:
Chirp.objects.all()[:-2]

AssertionError: Negative indexing is not supported.

In [21]:
Chirp.objects.all()[::2]

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>,
 <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>,
 <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>]

In [23]:
Chirp.objects.all()[::-1]

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>,
 <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>,
 <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>,
 <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>,
 <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>,
 <Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

In [24]:
Chirp.objects.all()[1:5]

[<Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>]

# Filters on Fields

In [25]:
chirp = Chirp.objects.get(pk=1)

In [26]:
chirp.title

'First Chirp'

In [27]:
Chirp.objects.filter(title__exact='First Chirp')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

In [28]:
Chirp.objects.filter(title__exact='First chirp')

[]

In [29]:
Chirp.objects.filter(title__iexact='First chirp')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

In [30]:
Chirp.objects.filter(message__contains='first')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

In [31]:
Chirp.objects.filter(message__startswith='This')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

# Spanning Relationships

In [32]:
Chirp.objects.filter(author__username='adam')

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

In [33]:
Chirp.objects.filter(author__username__contains='ad')

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

# F() ing objects

This allows you to filter based on data in each model as the filter loops through.

This can't work because the system doesn't think the variable author__username is undefined

In [34]:
Chirp.objects.filter(message__icontains=author__username)

NameError: name 'author__username' is not defined

In [35]:
Chirp.objects.filter(message__icontains=F('author__username'))

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

# PK Shortcut

In [36]:
Chirp.objects.get(pk=1)

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

In [37]:
Chirp.objects.get(id=1)

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

# Q() objects

In [38]:
Chirp.objects.filter(Q(message__startswith='First') | Q(message__startswith='This'))

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>]

# Order By

In [40]:
Chirp.objects.all().order_by('posted_at')

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>]

In [41]:
Chirp.objects.all().order_by('-posted_at')

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

In [42]:
Chirp.objects.all().order_by('-posted_at', 'message')

[<Chirp: Author: adam, Message: Adams first chirp, Posted at:2015-10-21 18:40:39.426578+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>]

# Raw for when Django can't cut it

In [43]:
Chirp.objects.raw('''SELECT * FROM chirp_chirp''')[0]

<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>

# Aggregates

In [44]:
jeff = User.objects.get(pk=1)

In [45]:
dir(jeff)

['DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 'REQUIRED_FIELDS',
 'USERNAME_FIELD',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_manager',
 '_check_column_name_clashes',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_ordering',
 '_check_swappable',
 '_check_unique_together',
 '_default_manager',
 '_deferred',
 '_do_insert',
 '_do_update',
 '_get_FIELD_display',
 '_get_next_or_previous_by_FIELD',
 '_get_next_or_previous_in_order',
 '_get_pk_val',
 '_get_unique_checks',
 '_

In [49]:
user = User.objects.annotate(number_of_chirps=Count('chirp'))[0]

In [50]:
user.number_of_chirps

5

In [51]:
user.chirp_set.all()

[<Chirp: Author: jeff, Message: This is my first chirp, Posted at:2015-10-21 17:43:13.289557+00:00>, <Chirp: Author: jeff, Message: Four Score and 7 years ago, Posted at:2015-10-21 17:46:18.178878+00:00>, <Chirp: Author: jeff, Message: We the people hold these truths to be self evident, Posted at:2015-10-21 17:46:50.746476+00:00>, <Chirp: Author: jeff, Message: First chirp from a shell, Posted at:2015-10-21 18:28:08.561673+00:00>, <Chirp: Author: jeff, Message: First chirp with create(), Posted at:2015-10-21 18:30:21.249537+00:00>]